In [9]:
import requests
import json
from mstrio.connection import Connection
from mstrio.api import reports
from mstrio.project_objects import Report


def open_Instance(conn, report_id):
    rep_instance = reports.report_instance(connection=conn, report_id=report_id)
    return rep_instance.json()["instanceId"]

def report_df(conn, report_id, instance_id):
    i_po_Report = Report(connection=conn, id=report_id, instance_id=instance_id)
    return i_po_Report.to_dataframe()

def get_report_sql(conn,report_id,instance_id):
    url=f'{conn.base_url}/api/v2/reports/{report_id}/instances/{instance_id}/sqlView'
    rep_sql=conn.get(url)
    return rep_sql

def set_inst_prompt_ans(conn, report_id, instance_id, prompt_answ):
    prompt_answ_url = f'{conn.base_url}/api/reports/{report_id}/instances/{instance_id}/prompts/answers'
    ret_prompt_ans = conn.put(prompt_answ_url, data=prompt_answ)
    return ret_prompt_ans

In [7]:
base_url="https://srvhclv100473.koeln.ads.risnet.de:8443/MicroStrategyLibrary"
username=""
password=""

project_id="B7CA92F04B9FAE8D941C3E9B7E0CD754"

conn = Connection(base_url=base_url,username=username,password=password,project_id=project_id)
conn.headers['Content-type'] = "application/json"

Connection to MicroStrategy Intelligence Server has been established.


In [46]:
report_id="09904D64418376362BCFDD86D03F8AF5"
element_pk=5
attribute_id="8D679D4B11D3E4981000E787EC6DE8A4"
element_ans= [{'id': f'h5;8D679D4B11D3E4981000E787EC6DE8A4'}]
prompt_id="38FB260A4099305D4220B485B927D84E"
prompt_answ={
      "id": prompt_id,
      "type": "ELEMENTS",
      "answers":  element_ans
    }
prompt_answ={"prompts":[prompt_answ]}
prompt_answ=json.dumps(prompt_answ)

instance_id=open_Instance(conn=conn, report_id=report_id)
T=set_inst_prompt_ans(conn=conn, report_id=report_id, instance_id=instance_id, prompt_answ=prompt_answ)
print(T)
rep_sql=get_report_sql(conn=conn, report_id=report_id, instance_id=instance_id)
pa_rep_df.json()

<Response [204]>


{'sqlStatement': 'select\t"a12"."region_id" AS "region_id",\n\tmax("a13"."region_name") AS "region_name0",\n\t(sum("a11"."tot_dollar_sales") - sum("a11"."tot_cost")) AS "WJXBFS1"\nfrom\t"public"."day_ctr_sls"\t"a11"\n\tjoin\t"public"."lu_call_ctr"\t"a12"\n\t  on \t("a11"."call_ctr_id" = "a12"."call_ctr_id")\n\tjoin\t"public"."lu_region"\t"a13"\n\t  on \t("a12"."region_id" = "a13"."region_id")\nwhere\t"a12"."region_id" in (5)\ngroup by\t"a12"."region_id"\n\n[Analytical engine calculation steps:\n\t1.  Perform cross-tabbing\n]\n'}

In [48]:
SQL_resp=get_report_sql(conn=conn,report_id=report_id,instance_id=instance_id)
sql_t=SQL_resp.json()
query=sql_t["sqlStatement"]

readable_query = query.replace('\t', '    ').replace('\n', '\n')

# Print the formatted query
print(readable_query)

select    "a12"."region_id" AS "region_id",
    max("a13"."region_name") AS "region_name0",
    (sum("a11"."tot_dollar_sales") - sum("a11"."tot_cost")) AS "WJXBFS1"
from    "public"."day_ctr_sls"    "a11"
    join    "public"."lu_call_ctr"    "a12"
      on     ("a11"."call_ctr_id" = "a12"."call_ctr_id")
    join    "public"."lu_region"    "a13"
      on     ("a12"."region_id" = "a13"."region_id")
where    "a12"."region_id" in (5)
group by    "a12"."region_id"

[Analytical engine calculation steps:
    1.  Perform cross-tabbing
]



In [ ]:
url="http://85.214.60.83:8080/MicroStrategyLibrary/api/cubes/72B4E8CD45561AAA899FF993EB3B0A91/attributes/8D679D4A11D3E4981000E787EC6DE8A4/elements?offset=0&limit=1000"
conn.get(url)